# 02 - Drift Detection
Generate and view drift report.

In [ ]:
import dagshub
dagshub.init(repo_owner='yahiaehab10', repo_name='MLflow_demo_MF', mlflow=True)
from src.drift_detection import save_baseline, generate_drift_report

# Set paths
base = '../data/drift_baseline/data.csv'
curr = '../data/processed/data.csv'
report = 'drift_report.html'

# Save baseline and generate report
save_baseline(curr, base)
generate_drift_report(base, curr, report)
print('Drift report generated at', report)